In [1]:
import pandas as pd
import os
import numpy as np
import epiweeks
import datetime
import requests
import zipfile
import csv
from contextlib import ExitStack
import progressbar as pbar
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
import warnings
from itertools import product
warnings.filterwarnings('ignore')
from openpyxl import load_workbook
from scipy import optimize

In [2]:
def prepare_weekly_covid_deaths(deaths, region: str):
    daily_regional_deaths = deaths[deaths['location_name'] == region].reset_index(drop=True)
    daily_regional_deaths = daily_regional_deaths.pivot(values='value', columns='date', index='Age group').fillna(0)
    daily_regional_deaths[daily_regional_deaths < 0] = 0

    weekly_regional_deaths = daily_regional_deaths.copy()
    [str(c) for c in daily_regional_deaths.columns]
    weekly_regional_deaths.columns = [epiweeks.Week.fromdate(datetime.date(year=int(str(c)[:4]), month=int(str(c)[5:7]), day=int(str(c)[8:10]))).isoformat() for c in daily_regional_deaths.columns]           
    t = weekly_regional_deaths.transpose()
    
    weekly_regional_deaths = t.groupby(t.index).sum().transpose()
    return daily_regional_deaths, weekly_regional_deaths

In [3]:
def prepare_weekly_covid_cases(cases, region: str):
    daily_regional_cases = cases[cases['location_name'] == region].reset_index(drop=True)
    daily_regional_cases = daily_regional_cases.pivot(values='value', index='Age group', columns='date').fillna(0)
    daily_regional_cases[daily_regional_cases < 0] = 0

    weekly_regional_cases = daily_regional_cases.copy()
    weekly_regional_cases.columns = [epiweeks.Week.fromdate(datetime.date(year=int(str(c)[:4]), month=int(str(c)[5:7]), day=int(str(c)[8: 10]))).isoformat() for c in daily_regional_cases.columns]
    t = weekly_regional_cases.transpose()
    weekly_regional_covid_cases = t.groupby(t.index).sum().transpose()
    return weekly_regional_covid_cases

In [4]:
def calculate_correction_due_to_aging(prior_populations, population, region: str):
    correction_due_to_aging = {}
    for year in prior_populations.keys():
        x = population.copy()
        x.iloc[:, 1:] = (prior_populations[year].iloc[:, 1:] / population.iloc[:, 1:])
        correction_due_to_aging[year] = {age_group: float(x[x['Age group'] == age_group][region]) for age_group in x['Age group'].unique()}
    return correction_due_to_aging

In [5]:
def calculate_reference_max_deaths(region, correction_due_to_aging, total_deaths):
    total_deaths = total_deaths[total_deaths['Year'] > str(2015)]
    max_deaths_correct = total_deaths[np.logical_and(total_deaths['Region'] == region, total_deaths['Year'] < '2020')].drop(columns=['Region']).reset_index(drop=True)
    max_deaths_correct['Multiplier'] = max_deaths_correct[['Year', 'Age group']].apply(lambda x: correction_due_to_aging[int(x[0])][x[1]], axis=1)
    for col in range(1, 52 + 1):
        max_deaths_correct[col] = max_deaths_correct[col] / max_deaths_correct['Multiplier']
    max_deaths_correct = max_deaths_correct.groupby('Age group').apply(max)[max_deaths_correct.columns[2:-2]]
    # max_deaths_correct = max_deaths_correct.groupby('Age group').apply(np.mean)[max_deaths_correct.columns[2:-2]]
    max_deaths_correct.columns = [f'W{i:02d}' for i in range(1, len(max_deaths_correct.columns) + 1)]
    y2020 = max_deaths_correct.add_prefix('2020')
    y2021 = max_deaths_correct.add_prefix('2021')
    y2022 = max_deaths_correct.add_prefix('2022')
    y2020['2020W53'] = np.maximum(max_deaths_correct['W01'], max_deaths_correct['W52'])
    max_deaths_concatenated = pd.concat([y2020, y2021, y2022], axis=1)
    max_deaths_concatenated[weekly_regional_deaths.columns]
    return max_deaths_concatenated

In [6]:
def calculate_overall_covid_deaths(total_deaths, region):
    if country == 'Sweden':
        overall_deaths = total_deaths[np.logical_and(total_deaths['Region'] == region, total_deaths['Year'] >= str(2020))].drop(columns=['Region']).sort_values(['Year', 'Age group']).reset_index(drop=True)
        y2020 = overall_deaths[overall_deaths['Year'] == '2020'].reset_index(drop=True)[overall_deaths.columns[1:-1]]
        y2020.columns = ['Age group'] + [f'2020W{i:02d}' for i in range(1, 53 + 1)]
        y2021 = overall_deaths[overall_deaths['Year'] == '2021'].reset_index(drop=True)[overall_deaths.columns[1:-2]]
        y2021.columns = ['Age group'] + [f'2021W{i:02d}' for i in range(1, 52 + 1)]
        y2022 = overall_deaths[overall_deaths['Year'] == '2022'].reset_index(drop=True)[overall_deaths.columns[1:-2]]
        y2022.columns = ['Age group'] + [f'2022W{i:02d}' for i in range(1, 52 + 1)]
        overall_deaths = pd.concat([y2020[y2020.columns[1:]], y2021[y2021.columns[1:]], y2022[y2022.columns[1:]]], axis=1)
        overall_deaths.index = y2020['Age group']
        overall_deaths = overall_deaths.sort_index()
    else:
        overall_deaths = total_deaths[np.logical_and(total_deaths['Region'] == region, total_deaths['Year'] >= str(2020))].drop(columns=['Region']).sort_values(['Year', 'Age group']).reset_index(drop=True)
        y2020 = overall_deaths[overall_deaths['Year'] == '2020'].reset_index(drop=True)[overall_deaths.columns[1:]]
        y2020.columns = ['Age group'] + [f'2020W{i:02d}' for i in range(1, 53 + 1)]
        y2021 = overall_deaths[overall_deaths['Year'] == '2021'].reset_index(drop=True)[overall_deaths.columns[1:-1]]
        y2021.columns = ['Age group'] + [f'2021W{i:02d}' for i in range(1, 52 + 1)]
        y2022 = overall_deaths[overall_deaths['Year'] == '2022'].reset_index(drop=True)[overall_deaths.columns[1:-1]]
        y2022.columns = ['Age group'] + [f'2022W{i:02d}' for i in range(1, 52 + 1)]
        overall_deaths = pd.concat([y2020[y2020.columns[1:]], y2021[y2021.columns[1:]], y2022[y2022.columns[1:]]], axis=1)
        overall_deaths.index = y2020['Age group']
        overall_deaths = overall_deaths.sort_index()
    return overall_deaths

In [7]:
def plot_corrected_covid_deaths():
    if not os.path.exists(f'{result_dir_path}/{country}/{region}/gamma={gamma:.2f}'):
        os.mkdir(f'{result_dir_path}/{country}/{region}/gamma={gamma:.2f}')
    for age_group in max_deaths.index:
        fig, axs = plt.subplots(1, 2, figsize=(30,20))
        
        weekly_regional_deaths.loc[age_group, :].T.plot(ax=axs[0])
        corrected_covid_deaths.loc[age_group, :].T.plot(ax=axs[0])
        additional_deaths_not_attributed_to_covid.loc[age_group, :].T.plot(ax=axs[0])
        axs[0].legend(['covid deaths (reported)', f'corrected covid deaths {gamma:.2f}', 'excess deaths'])
        axs[0].set_title(f'Age groups: {age_group} ({country} - {region})')
        axs[0].set_xlabel('week')
        axs[0].set_ylabel('deaths')
        axs[0].grid()
        
        weekly_regional_deaths.loc[age_group, :].cumsum().T.plot(ax=axs[1])
        corrected_covid_deaths.loc[age_group, :].cumsum().T.plot(ax=axs[1])
        additional_deaths_not_attributed_to_covid.loc[age_group, :].cumsum().T.plot(ax=axs[1])
        axs[1].legend(['covid deaths (reported)', f'corrected covid deaths {gamma:.2f}', 'excess deaths'])
        axs[1].set_title(f'Age groups: {age_group} ({country} - {region})')
        axs[1].set_xlabel('week')
        axs[1].set_ylabel('deaths')
        axs[1].grid()

        plt.savefig(f'{result_dir_path}/{country}/{region}/gamma={gamma:.2f}/{age_group}.png')
        plt.close('all')

In [8]:
def plot_cumulative_deaths():
    if not os.path.exists(f'{result_dir_path}/{country}/{region}/Total deaths with max deaths line'):
        os.mkdir(f'{result_dir_path}/{country}/{region}/Total deaths with max deaths line')

    max_deaths_new = pd.DataFrame(max_deaths.iloc[:-1].sum(), index=max_deaths.columns, columns=['Total']).T

    for age_group in max_deaths.index:
        fig, axs = plt.subplots(figsize=(15, 10))
        
        y2020 = total_deaths[np.logical_and(total_deaths['Year'] == '2020', total_deaths['Age group'] == age_group)].drop(['Year', 'Region', 'Age group'],  axis='columns').iloc[:, :]
        y2020.columns = y2020.columns.map(lambda x: f'2020W{x:02d}')
        y2021 = total_deaths[np.logical_and(total_deaths['Year'] == '2021', total_deaths['Age group'] == age_group)].drop(['Year', 'Region', 'Age group'],  axis='columns').iloc[:, :-1]
        y2021.columns = y2021.columns.map(lambda x: f'2021W{x:02d}')
        y2022 = total_deaths[np.logical_and(total_deaths['Year'] == '2022', total_deaths['Age group'] == age_group)].drop(['Year', 'Region', 'Age group'],  axis='columns').iloc[:, :-1]
        y2022.columns = y2022.columns.map(lambda x: f'2022W{x:02d}')
        total_deaths_line = pd.concat([y2020, y2021, y2022], axis=1)
        
        total_deaths_line.loc[:, weekly_regional_deaths.loc[age_group, :].index].T.plot(ax=axs)
        max_deaths.loc[age_group, weekly_regional_deaths.loc[age_group, :].index].T.plot(ax=axs)
        (max_deaths.loc[age_group, weekly_regional_deaths.loc[age_group, :].index] + weekly_regional_deaths.loc[age_group, :].to_numpy()).T.plot(ax=axs)
        axs.legend(['Overall deaths', 'Max deaths', 'Max deaths + covid deaths (reported)'])
        if age_group == 'TOTAL':
            max_deaths_new.loc[:, weekly_regional_deaths.loc[age_group, :].index].T.plot(ax=axs)
            axs.legend(['Overall deaths', 'Max deaths', 'Max deaths + covid deaths (reported)', 'Max deaths - sum(max)'])
        
        axs.set_title(f'Age groups: {age_group} ({country} - {region})')
        axs.set_xlabel('week')
        axs.set_ylabel('deaths')
        axs.grid()

        plt.savefig(f'{result_dir_path}/{country}/{region}/Total deaths with max deaths line/{age_group}.png')
        plt.close('all')

In [9]:
today = datetime.date.today().strftime("%d-%m-%Y")
# today = '06-03-2022'
data_dir_path = f'data/World/{today}'
result_dir_path = f'result/World/{today}'
if not os.path.exists(f'{result_dir_path}'):
    os.mkdir(f'{result_dir_path}')
with open(f'{data_dir_path}/eurostat_countries.json', 'r') as file:
    countries = json.load(file)

In [10]:
format_label = pbar.FormatCustomText('Country: %(country)s Running: %(running)s   ', dict(country='start', running='start'))
widgets = [pbar.ETA(), ' ', pbar.Timer(), ' ', pbar.Percentage(format=' (%(percentage)3d%%) '), '  ', format_label]
bar = pbar.ProgressBar(widgets=widgets, maxval=len(countries))
bar.start()

with open(f'{data_dir_path}/ifr_5_years_age_group.json', 'r') as file:
    IFRs = json.load(file)

df_method_1 = pd.DataFrame(columns=['covid deaths gamma=0 (fraction of population)[%]', 'covid deaths gamma=0.8 (fraction of population)[%]', 'covid deaths gamma=1 (fraction of population)[%]',
                                    'covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]',
                                    'covid cases (fraction of population)[%]', 'IFR'])

df_method_2 = pd.DataFrame(columns=['covid deaths gamma=0 (fraction of population)[%]', 'covid deaths gamma=0.8 (fraction of population)[%]', 'covid deaths gamma=1 (fraction of population)[%]',
                                    'covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]',
                                    'covid cases (fraction of population)[%]', 'IFR'])

df_method_3 = pd.DataFrame(columns=['covid deaths gamma=0 (fraction of population)[%]', 'covid deaths gamma=0.8 (fraction of population)[%]', 'covid deaths gamma=1 (fraction of population)[%]',
                                    'covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]',
                                    'covid cases (fraction of population)[%]', 'IFR'])

df_method_4 = pd.DataFrame(columns=['covid deaths gamma=0 (fraction of population)[%]', 'covid deaths gamma=0.8 (fraction of population)[%]', 'covid deaths gamma=1 (fraction of population)[%]',
                                    'covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]',
                                    'covid cases (fraction of population)[%]', 'IFR'])

df_method_5 = pd.DataFrame(columns=['covid deaths gamma=0 (fraction of population)[%]', 'covid deaths gamma=0.8 (fraction of population)[%]', 'covid deaths gamma=1 (fraction of population)[%]',
                                    'covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]',
                                    'covid cases (fraction of population)[%]', 'IFR'])

df_method_6 = pd.DataFrame(columns=['covid deaths gamma=0 (fraction of population)[%]', 'covid deaths gamma=0.8 (fraction of population)[%]', 'covid deaths gamma=1 (fraction of population)[%]',
                                    'covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]',
                                    'covid cases (fraction of population)[%]', 'IFR'])

df_method_7 = pd.DataFrame(columns=['covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]'])

df_method_8 = pd.DataFrame(columns=['covid deaths gamma=0 (fraction of population)[%]', 'covid deaths gamma=0.8 (fraction of population)[%]', 'covid deaths gamma=1 (fraction of population)[%]',
                                    'covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]',
                                    'covid cases (fraction of population)[%]', 'IFR'])

for country_index, (country, regions) in enumerate(countries.items()):
#     if country != 'Germany':
#         continue
    if country not in [
        'Austria',
        'Belgium',
        'Bulgaria',
        'Switzerland',
        'Czechia',
        'Germany',
        'Denmark',
        'Greece',
        'Spain',
        'Finland',
        'France',
        'Croatia',
        'Hungary',
        'Italy',
        'Latvia',
        'Montenegro',
        'Netherlands',
        'Norway',
        'Portugal',
        'Romania',
        'Sweden',
        'Slovenia',
        'Slovakia',
        'United Kingdom',
        
        
    ]:
        continue

    format_label.update_mapping(country=country)
    bar.update()
    
    # Load cases
    format_label.update_mapping(running='Loading cases' + ' ' * 30)
    bar.update()
    cases = pd.read_csv(f'{data_dir_path}/{country}/covid_cases.csv', index_col=0).reset_index(drop=True)
    cases = cases.append(cases.groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('TOTAL')
    cases = cases.append(cases[cases['Age group'].isin(['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49'])].groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('00-49')
    cases = cases.append(cases[cases['Age group'].isin(['55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+', '50-54'])].groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('50+')
    cases = cases.append(cases[cases['Age group'].isin(['80-84', '85+'])].groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('80+')
    cases = cases[~cases['Age group'].isin(['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '80-84', '85+'])].reset_index(drop=True)
    
    # Load deaths
    format_label.update_mapping(running='Loading deaths' + ' ' * 30)
    bar.update()
    deaths = pd.read_csv(f'{data_dir_path}/{country}/covid_deaths.csv', index_col=0).reset_index(drop=True)
    deaths = deaths.append(deaths.groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('TOTAL')
    deaths = deaths.append(deaths[deaths['Age group'].isin(['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49'])].groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('00-49')
    deaths = deaths.append(deaths[deaths['Age group'].isin(['55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+', '50-54'])].groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('50+')
    deaths = deaths.append(deaths[deaths['Age group'].isin(['80-84', '85+'])].groupby(['date', 'location', 'location_name']).sum().reset_index()).fillna('80+')
    deaths = deaths[~deaths['Age group'].isin(['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '80-84', '85+'])].reset_index(drop=True)
    

    # Load population
    format_label.update_mapping(running='Loading population' + ' ' * 30)
    bar.update()
    population = pd.read_csv(f'{data_dir_path}/{country}/old_population.csv')
    population = population[~population['Age group'].isin(['UNK', 'Y_OPEN'])]
    population['Age group'] = population['Age group'].replace({'TOTAL': 'YTOTAL', 'Y_LT1': 'Y0'}).apply(lambda x: x[1:])
    new_df = pd.DataFrame(columns=['Year', 'Region', 'Age group', 'Sex', 'Value'])
    new_df = new_df.append(population[population['Age group'] == 'TOTAL'])
    for start, stop in zip([0, 50, 55, 60, 65, 70, 75, 80, 50, 50], [50, 55, 60, 65, 70, 75, 80, 100, 65, 100]): 
        new_df = new_df.append(population[population['Age group'].isin([str(i) for i in range(start, stop)])].groupby(['Year', 'Region', 'Sex']).sum().reset_index())
        new_df['Age group'] = new_df['Age group'].fillna(f'{start:02d}-{stop-1:02d}')
    new_df['Age group'] = new_df['Age group'].replace({'80-99': '80+', '50-99': '50+'})
    population = new_df.reset_index(drop=True)
    prior_populations = dict()
    for year in range(2015, 2020):
        df = pd.DataFrame(columns=['Age group'] + [regions])
        df['Age group'] = population['Age group'].unique()
        for region in [regions]:
            condition = np.logical_and(population['Region'] == region, population['Year'] == year)
            condition = np.logical_and(condition, population['Sex'] == 'T')
            if len(population[condition]['Value']) < len(df['Age group']):
                df.loc[:, region] = 0
            else:
                df.loc[:, region] = list(population[condition]['Value'])
        prior_populations[year] = df.sort_values(by='Age group').reset_index().drop('index', axis=1)
    
    if country == 'United Kingdom':
        population = population[population['Year'] == 2019].reset_index(drop=True)
    else:
        population = population[population['Year'] == 2021].reset_index(drop=True)
    new_df = pd.DataFrame({'Age group': population['Age group'].unique()})
    for i, region in enumerate([regions]):
        new_df.insert(i + 1, region, list(population[np.logical_and(population['Region'] == region, population['Sex'] == 'T')]['Value']), True)
    new_df = new_df[~new_df['Age group'].isin(['UNK', 'Y_GE75', 'Y_GE80'])].sort_values('Age group').reset_index(drop=True)
    population = new_df
    
    bar.update()
    
    # Load and prepare total deaths
    format_label.update_mapping(running='Loading total deaths' + ' ' * 30)
    bar.update()
    age_group_translator = {'Y_LT5': '00-04', 'Y5-9': '05-09', 'Y10-14': '10-14', 'Y15-19': '15-19', 'Y20-24': '20-24', 'Y25-29': '25-29', 'Y30-34': '30-34', 'Y35-39': '35-39', 'Y40-44': '40-44','Y45-49': '45-49',
                            'Y50-54': '50-54', 'Y55-59': '55-59', 'Y60-64': '60-64', 'Y65-69': '65-69', 'Y70-74': '70-74', 'Y75-79': '75-79', 'Y80-84': '80-84', 'Y_GE85': '85+', 'Y85-89': '85-89', 'Y_GE90': '90+',}

    total_deaths = pd.read_csv(f'{data_dir_path}/{country}/old_deaths.csv')
    total_deaths = total_deaths[total_deaths['Region'] == regions]
    total_deaths = total_deaths[total_deaths['Sex'] == 'T'].reset_index(drop=True)
    total_deaths = total_deaths[total_deaths['Year'].apply(lambda x: x[:4]) >= str(2015)].reset_index(drop=True)
    new_df = pd.DataFrame(columns=['Year', 'Region', 'Age group'] + list(np.arange(1, 54)))
    new_df = new_df.append(total_deaths[total_deaths['Sex'] == 'T'])
    new_df['Year'] = new_df['Year'].apply(lambda x: x[:4])
    new_df = new_df.drop(columns=['Sex', 'Value'], axis=1)
    new_df = new_df.drop_duplicates()
    
    for i in range(len(total_deaths)):
        region = total_deaths.iloc[i]['Region']
        age_group = total_deaths.iloc[i]['Age group']
        year = total_deaths.iloc[i]['Year'][:4]
        week = int(total_deaths.iloc[i]['Year'][6:])
        x = np.logical_and(new_df['Region'] == region, new_df['Year'] == year)
        x = np.logical_and(x, new_df['Year'] >= str(2015))
        x = np.logical_and(x, new_df['Age group'] == age_group)
        new_df.loc[x, week] = total_deaths.iloc[i]['Value']
    total_deaths = new_df[new_df['Year'] >= str(2015)]
    total_deaths = total_deaths.drop(total_deaths.loc[total_deaths['Age group']=='UNK'].index)
    total_deaths = total_deaths.drop(total_deaths.loc[total_deaths['Age group']=='Y_GE75'].index)
    total_deaths = total_deaths.drop(total_deaths.loc[total_deaths['Age group']=='Y_GE80'].index).fillna(0)
    total_deaths_5_years_age_group = total_deaths.copy()
    total_deaths.replace(age_group_translator, inplace=True)
    total_deaths = total_deaths.append(total_deaths[total_deaths['Age group'].isin(['85-89', '90+'])].groupby(['Year', 'Region']).sum().reset_index())
    total_deaths = total_deaths.fillna({'Age group': '85+'})
    total_deaths = total_deaths[~total_deaths['Age group'].isin(['85-89', '90+'])].sort_values('Year').reset_index(drop=True)
    
    total_deaths['Age group'].unique()
    total_deaths = total_deaths.append(total_deaths[total_deaths['Age group'].isin(['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49'])].groupby(['Year', 'Region']).sum().reset_index()).fillna('00-49')
    total_deaths = total_deaths.append(total_deaths[total_deaths['Age group'].isin(['55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85+', '50-54'])].groupby(['Year', 'Region']).sum().reset_index()).fillna('50+')
    total_deaths = total_deaths.append(total_deaths[total_deaths['Age group'].isin(['80-84', '85+'])].groupby(['Year', 'Region']).sum().reset_index()).fillna('80+')
    total_deaths = total_deaths[~total_deaths['Age group'].isin(['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '85+', '80-84'])].sort_values('Year').reset_index(drop=True)
    bar.update()
    
    df = pd.DataFrame(columns= ['Year', 'Region', 'Age group'] + list(range(1, 53+1)))
    for region in [regions]:
        # Prepare dir
        format_label.update_mapping(running='Prepare dir' + ' ' * 30)
        bar.update()
        if not os.path.exists(f'{result_dir_path}/{country}'):
            os.mkdir(f'{result_dir_path}/{country}')  
            
        # Simulation
        format_label.update_mapping(running='Simulation' + ' ' * 30)
        bar.update()
        daily_regional_deaths, weekly_regional_deaths = prepare_weekly_covid_deaths(deaths, region)
        weekly_regional_cases = prepare_weekly_covid_cases(cases, region)

        correction_due_to_aging = calculate_correction_due_to_aging(prior_populations, population, region)
        max_deaths = calculate_reference_max_deaths(region, correction_due_to_aging, total_deaths)
        overall_deaths = calculate_overall_covid_deaths(total_deaths, region).apply(pd.to_numeric)
        
        overall_minus_max = np.maximum(0, overall_deaths - max_deaths).loc[:, :weekly_regional_deaths.columns[-1]]
        additional_deaths_not_attributed_to_covid = np.maximum(0, overall_minus_max[weekly_regional_deaths.columns] - weekly_regional_deaths).fillna(0)
        
        # Plot results
        format_label.update_mapping(country=country, running='Ploting result: weekly covid deaths' + ' ' * 30)
        bar.update()
        fig, axs = plt.subplots(1, 2, figsize=(30,10))
        weekly_regional_deaths.T.plot(ax=axs[0])
        weekly_regional_deaths.cumsum(axis=1).T.plot(ax=axs[1])
        axs[0].set(title=f'Weekly Covid deaths recorded by week in age groups ({country} - {region})', xlabel='week', ylabel='deaths')
        axs[1].set(title=f'Cumulative Covid deaths recorded by week in age groups ({country} - {region})', xlabel='week', ylabel='deaths')
        axs[0].grid()
        axs[1].grid()
        plt.savefig(f'{result_dir_path}/{country}/cumulative_covid_deaths_reported.png')
        plt.close('all')

        
        format_label.update_mapping(country=country, running='Ploting result: weekly covid cases' + ' ' * 30)
        bar.update()
        fig, axs = plt.subplots(1, 2, figsize=(30,10))
        weekly_regional_cases.T.plot(ax=axs[0])
        weekly_regional_cases.cumsum(axis=1).T.plot(ax=axs[1])
        axs[0].set(title=f'Weekly Covid cases recorded by week in age groups ({country} - {region})', xlabel='week', ylabel='cases')
        axs[1].set(title=f'Cumulative Covid cases recorded by week in age groups ({country} - {region})', xlabel='week', ylabel='cases')
        axs[0].grid()
        axs[1].grid()
        plt.savefig(f'{result_dir_path}/{country}/cumulative_covid_cases_reported.png')
        plt.close('all')
    
    

        format_label.update_mapping(country=country, running='Ploting result: darkfigure' + ' ' * 30)
        bar.update()
        if country == 'United Kingdom':
            weekly_regional_cases = weekly_regional_cases.iloc[:, :-8]
        window_list = [3]
        gamma_list = [0, 0.8, 1]
        
        df_method_1 = df_method_1.append(pd.DataFrame(index=[country]))
        df_method_2 = df_method_2.append(pd.DataFrame(index=[country]))
        df_method_3 = df_method_3.append(pd.DataFrame(index=[country]))
        df_method_4 = df_method_4.append(pd.DataFrame(index=[country]))
        df_method_5 = df_method_5.append(pd.DataFrame(index=[country]))
        df_method_6 = df_method_6.append(pd.DataFrame(index=[country]))
        df_method_7 = df_method_7.append(pd.DataFrame(index=[country]))
        df_method_8 = df_method_8.append(pd.DataFrame(index=[country]))
        week = '2021W22'
        
        
        for gamma, window in product(gamma_list, window_list):
            format_label.update_mapping(country=country, running=f'Ploting result: darkfigure {gamma=}, {window=}' + ' ' * 30)
            bar.update()
            
            
            age_group = 'TOTAL'
            fig, ax = plt.subplots(3, 2, figsize=(30,20))

            weekly_regional_deaths_window = weekly_regional_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()
            overall_minus_max_window = np.maximum(0, overall_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean() - max_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()).loc[:, :weekly_regional_deaths.columns[-1]]
            additional_deaths_not_attributed_to_covid_window = np.maximum(0, overall_minus_max_window - weekly_regional_deaths_window).fillna(0)
            corrected_covid_deaths_window = additional_deaths_not_attributed_to_covid_window * gamma + weekly_regional_deaths_window

            overall_minus_max = np.maximum(0, overall_deaths - max_deaths).loc[:, weekly_regional_deaths.columns]
            additional_deaths_not_attributed_to_covid = np.maximum(0, overall_minus_max - weekly_regional_deaths).fillna(0)
            corrected_covid_deaths = additional_deaths_not_attributed_to_covid * gamma + weekly_regional_deaths

            additional_deaths_not_attributed_to_covid_window.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[0, 0], label=f'excess {gamma=:.2f}', style='b')
            additional_deaths_not_attributed_to_covid.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[0, 1], label=f'excess {gamma=:.2f}', style='b')


            weekly_regional_deaths_window.loc[age_group].T.plot(ax=ax[1, 0], style='g', label='covid')
            corrected_covid_deaths_window.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[1, 0], label=f'covid + excess {gamma=:.2f}', style='b')

            weekly_regional_deaths.loc[age_group].T.plot(ax=ax[1, 1], style='g', label='covid')
            corrected_covid_deaths.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[1, 1], label=f'covid + excess {gamma=:.2f}', style='b')

            weekly_regional_deaths_window.cumsum(axis=1).loc[age_group].T.plot(ax=ax[2, 0], label=f'covid', style='g')
            corrected_covid_deaths_window.cumsum(axis=1).loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[2, 0], label=f'covid + excess {gamma=:.2f}', style='b')
            corrected_covid_deaths.cumsum(axis=1).loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[2, 0], label=f'covid + excess (without mean) {gamma=:.2f}', style='--r')

            weekly_regional_deaths.cumsum(axis=1).loc[age_group].T.plot(ax=ax[2, 1], label='covid', style='g')
            corrected_covid_deaths.cumsum(axis=1).loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[2, 1], label=f'covid + excess {gamma=:.2f}', style='b')

            ax[0, 1].legend()
            ax[0, 0].legend()
            ax[1, 1].legend()
            ax[1, 0].legend()
            ax[2, 1].legend()
            ax[2, 0].legend()

            ax[0, 0].set_ylim([0, round(np.max(np.max(overall_minus_max)), -2) * 1.1])
            ax[0, 1].set_ylim([0, round(np.max(np.max(overall_minus_max)), -2) * 1.1])
            ax[1, 0].set_ylim([0, round(np.max(np.max(weekly_regional_deaths)), -2) * 1.1])
            ax[1, 1].set_ylim([0, round(np.max(np.max(weekly_regional_deaths)), -2) * 1.1])
            ax[2, 0].set_ylim([0, round(np.max(np.max(corrected_covid_deaths.cumsum(axis=1))), -2) * 1.1])
            ax[2, 1].set_ylim([0, round(np.max(np.max(corrected_covid_deaths.cumsum(axis=1))), -2) * 1.1])

            ax[0, 0].set_title(f'excess deaths with mean {window=}')
            ax[0, 1].set_title('excess deahts without mean')
            ax[1, 0].set_title(f'covid deaths with mean {window=} (weekly)')
            ax[1, 1].set_title('covid deaths without mean (weekly)')
            ax[2, 0].set_title(f'covid deaths with mean {window=} (cumulative)')
            ax[2, 1].set_title('covid deaths without mean (cumulative)')

            ax[0, 1].grid()
            ax[0, 0].grid()
            ax[1, 1].grid()
            ax[1, 0].grid()
            ax[2, 1].grid()
            ax[2, 0].grid()
            
            ax[0, 0].set_xlabel('week')
            ax[0, 1].set_xlabel('week')
            ax[1, 0].set_xlabel('week')
            ax[1, 1].set_xlabel('week')
            ax[2, 0].set_xlabel('week')
            ax[2, 1].set_xlabel('week')
            
            ax[0, 0].set_ylabel('deaths')
            ax[0, 1].set_ylabel('deaths')
            ax[1, 0].set_ylabel('deaths')
            ax[1, 1].set_ylabel('deaths')
            ax[2, 0].set_ylabel('deaths')
            ax[2, 1].set_ylabel('deaths')
            if not os.path.exists(f'{result_dir_path}/{country}/Window comparation/'):
                os.mkdir(f'{result_dir_path}/{country}/Window comparation/')
            plt.savefig(f'{result_dir_path}/{country}/Window comparation/window={window}_gamma={gamma}.png')
            plt.close('all')
            bar.update()
            
            weekly_regional_deaths_window = weekly_regional_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()
            overall_minus_max_window = np.maximum(0, overall_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean() - max_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()).loc[:, :weekly_regional_deaths.columns[-1]]
            additional_deaths_not_attributed_to_covid_window = np.maximum(0, overall_minus_max_window - weekly_regional_deaths_window).fillna(0)
            corrected_covid_deaths_window = additional_deaths_not_attributed_to_covid_window * gamma + weekly_regional_deaths_window

            # Method 1
            df = weekly_regional_cases.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+', 'TOTAL'], :].rolling(window=window, axis=1, min_periods=1, center=True).mean().cumsum(axis=1)
            cases_based_on_ifr = corrected_covid_deaths_window.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].copy().cumsum(axis=1)
            cases_based_on_ifr = cases_based_on_ifr.append(pd.DataFrame(cases_based_on_ifr.sum(), columns=['TOTAL']).T)
            df_method_1[f'covid deaths gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.iloc[:-1, :].sum()[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)
            for age_group, ifr in IFRs[country].items():
                if age_group in ['45-64', '50-64', '50+']:
                    continue
                cases_based_on_ifr.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :].apply(lambda x: 100*x/ifr)
                df.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :] / df.loc[age_group, :]
                df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
                df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))
            df_method_1[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.iloc[:-1, :].sum()[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)   
            df_method_1['covid cases (fraction of population)[%]'][country] = float(weekly_regional_cases.cumsum(axis=1).iloc[-1, :][week] / population[population['Age group'] == 'TOTAL'][region].to_numpy()* 100)
            df_method_1['IFR'][country] = IFRs[country]['TOTAL']

            # Method 3
            df = weekly_regional_cases.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+', 'TOTAL'], :].rolling(window=window, axis=1, min_periods=1, center=True).mean().cumsum(axis=1)
            cases_based_on_ifr = corrected_covid_deaths_window.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].copy().cumsum(axis=1)
            cases_based_on_ifr = cases_based_on_ifr.append(pd.DataFrame(cases_based_on_ifr.sum(), columns=['TOTAL']).T)
            df_method_3[f'covid deaths gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.iloc[:-1, :].sum()[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)
            for age_group, ifr in IFRs[country].items():
                if age_group in ['45-64', '50-64', '50+']:
                    continue
                if age_group == '00-49':
                    df_60_64 = corrected_covid_deaths_window.copy().cumsum(axis=1).loc[['50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].sum().apply(lambda x: 100*x/IFRs[country]['50+'])
                    df.loc[age_group, :] = df_60_64 / df.loc[['50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].sum()
                    df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
                    df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))
                    cases_based_on_ifr.loc[age_group, :] = (weekly_regional_cases.rolling(window=window, axis=1, min_periods=1, center=True).mean().loc[age_group, :] * df.loc[age_group, :]).cumsum()
                else:                                                                                                            
                    cases_based_on_ifr.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :].apply(lambda x: 100*x/ifr)
                    df.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :] / df.loc[age_group, :]
                    df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
                    df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))   
            df_method_3[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.iloc[:-1, :].sum()[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)   
            df_method_3['covid cases (fraction of population)[%]'][country] = float(weekly_regional_cases.cumsum(axis=1).iloc[-1, :][week] / population[population['Age group'] == 'TOTAL'][region].to_numpy()* 100)
            df_method_3['IFR'][country] = IFRs[country]['TOTAL']

            # Method 4
            df = weekly_regional_cases.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+', 'TOTAL'], :].rolling(window=window, axis=1, min_periods=1, center=True).mean().cumsum(axis=1)
            cases_based_on_ifr = corrected_covid_deaths_window.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].copy().cumsum(axis=1)
            cases_based_on_ifr = cases_based_on_ifr.append(pd.DataFrame(cases_based_on_ifr.sum(), columns=['TOTAL']).T)
            df_method_4[f'covid deaths gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.iloc[:-1, :].sum()[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)
            for age_group, ifr in IFRs[country].items():
                if age_group in ['45-64', '50-64', '50+']:
                    continue
                if age_group == '00-49':
                    df_60_64 = corrected_covid_deaths_window.copy().cumsum(axis=1).loc[['50-54', '55-59', '60-64'], :].sum().apply(lambda x: 100*x/IFRs[country]['50-64'])
                    df.loc[age_group, :] = df_60_64 / df.loc[['50-54', '55-59', '60-64'], :].sum()
                    df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
                    df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))
                    cases_based_on_ifr.loc[age_group, :] = (weekly_regional_cases.rolling(window=window, axis=1, min_periods=1, center=True).mean().loc[age_group, :] * df.loc[age_group, :]).cumsum()
                else:                                                                                                            
                    cases_based_on_ifr.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :].apply(lambda x: 100*x/ifr)
                    df.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :] / df.loc[age_group, :]
                    df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
                    df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))
            df_method_8[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.iloc[-1, :][week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)   
            df_method_8['covid cases (fraction of population)[%]'][country] = float(weekly_regional_cases.cumsum(axis=1).iloc[-1, :][week] / population[population['Age group'] == 'TOTAL'][region].to_numpy()* 100)
            df_method_8['IFR'][country] = IFRs[country]['TOTAL']
            df_method_4[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.iloc[:-1, :].sum()[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)   
            df_method_4['covid cases (fraction of population)[%]'][country] = float(weekly_regional_cases.cumsum(axis=1).iloc[-1, :][week] / population[population['Age group'] == 'TOTAL'][region].to_numpy()* 100)
            df_method_4['IFR'][country] = IFRs[country]['TOTAL']

            # Method 2
            df = weekly_regional_cases.loc[['TOTAL'], :].rolling(window=window, axis=1, min_periods=1, center=True).mean().cumsum(axis=1)
            cases_based_on_ifr = corrected_covid_deaths_window.loc[['TOTAL'], :].copy().cumsum(axis=1)
            df_method_2[f'covid deaths gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)
            for age_group, ifr in IFRs[country].items():
                if age_group in ['45-64', '50-64', '50+', '00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+']:
                    continue                                                                                                          
                cases_based_on_ifr.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :].apply(lambda x: 100*x/ifr)
                df.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :] / df.loc[age_group, :]
                df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
                df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))   
            df_method_2[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr[week] / population[population['Age group'] == 'TOTAL'][region].to_numpy() * 100)
            df_method_2['covid cases (fraction of population)[%]'][country] = float(weekly_regional_cases.cumsum(axis=1).iloc[-1, :][week] / population[population['Age group'] == 'TOTAL'][region].to_numpy()* 100)
            df_method_2['IFR'][country] = IFRs[country]['TOTAL']

            # Method 5 and 6
            df = weekly_regional_cases.loc[['50+', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].rolling(window=window, axis=1, min_periods=1, center=True).mean().cumsum(axis=1)
            cases_based_on_ifr = corrected_covid_deaths_window.loc[['50+', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].copy().cumsum(axis=1)
            df_method_5[f'covid deaths gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.loc[['50+'], :][week] / population[population['Age group'] == '50+'][region].to_numpy() * 100)
            df_method_6[f'covid deaths gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.loc[['50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].sum()[week] / population[population['Age group'] == '50+'][region].to_numpy() * 100)
            for age_group, ifr in IFRs[country].items():
                if age_group in ['45-64', '50-64', 'TOTAL', '00-49']:
                    continue                                                                                                          
                cases_based_on_ifr.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :].apply(lambda x: 100*x/ifr)
                df.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :] / df.loc[age_group, :]
                df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
                df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))   
            df_method_5[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.loc[['50+'], :][week] / population[population['Age group'] == '50+'][region].to_numpy() * 100)
            df_method_6[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(cases_based_on_ifr.loc[['50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].sum()[week] / population[population['Age group'] == '50+'][region].to_numpy() * 100)
            df_method_5['covid cases (fraction of population)[%]'][country] = float(weekly_regional_cases.cumsum(axis=1).iloc[-1, :][week] / population[population['Age group'] == '50+'][region].to_numpy()* 100)
            df_method_5['IFR'][country] = IFRs[country]['50+']
            df_method_6['covid cases (fraction of population)[%]'][country] = float(weekly_regional_cases.cumsum(axis=1).iloc[-1, :][week] / population[population['Age group'] == '50+'][region].to_numpy()* 100)
            df_method_6['IFR'][country] = IFRs[country]['50+']
            bar.update()
            
            # Method 7
            def func(x, i):
                return i*x
            da = corrected_covid_deaths_window.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].copy().cumsum(axis=1)[week].to_numpy()
            pa = population[population['Age group'].isin(['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'])].reset_index(drop=True).drop(columns=['Age group']).T.to_numpy()[0] 
            pafa = pa * np.array([IFRs[country][key] / 100 for key in ['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+']]) / population[population['Age group'] == 'TOTAL'].reset_index(drop=True).drop(columns=['Age group']).T.to_numpy()[0]

            alpha = optimize.curve_fit(func, xdata = pafa, ydata = da)[0]
            
            df_method_7[f'covid cases by ifr gamma={gamma} (fraction of population)[%]'][country] = float(alpha / population[population['Age group'] == 'TOTAL'].reset_index(drop=True).drop(columns=['Age group']).T.to_numpy()[0] * 100)


        format_label.update_mapping(country=country, running=f'Ploting result: Death factor' + ' ' * 30)
        bar.update()
        
        for window in window_list:
            fig, axs = plt.subplots(1, 2, figsize=(30,10))
            for gamma in gamma_list:
                corrected_covid_deaths_window = additional_deaths_not_attributed_to_covid_window * gamma + weekly_regional_deaths_window
                corrected_covid_deaths_window.loc[['00-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+'], :].sum().cumsum().plot(ax=axs[0], label=f'gamma = {gamma}')
                corrected_covid_deaths_window.loc['TOTAL'].cumsum().plot(ax=axs[1], label=f'gamma = {gamma}')

            axs[0].legend()
            axs[0].set_xlabel('week')
            axs[0].set_ylabel('deaths')
            axs[0].legend()
            axs[0].grid()
            axs[0].set_title('Total corrected covid deaths by age group')
            axs[0].set_ylim([0, round(np.max(np.max(corrected_covid_deaths_window.cumsum(axis=1))), -2) * 1.1])

            axs[1].legend()
            axs[1].set_xlabel('week')
            axs[1].set_ylabel('deaths')
            axs[1].grid()
            axs[1].set_title('Total corrected covid deaths by total')
            axs[1].set_ylim([0, round(np.max(np.max(corrected_covid_deaths_window.cumsum(axis=1))), -2) * 1.1])

            if not os.path.exists(f'{result_dir_path}/{country}/Death factor/'):
                os.mkdir(f'{result_dir_path}/{country}/Death factor/')
            plt.savefig(f'{result_dir_path}/{country}/Death factor/window={window}.png')
            plt.close('all')
            bar.update()
        
        
        
    bar.update(country_index)
df_method_1.to_excel(f'{result_dir_path}/Method_1.xlsx')
df_method_2.to_excel(f'{result_dir_path}/Method_2.xlsx')
df_method_3.to_excel(f'{result_dir_path}/Method_3.xlsx')
df_method_4.to_excel(f'{result_dir_path}/Method_4.xlsx')
df_method_5.to_excel(f'{result_dir_path}/Method_5.xlsx')
df_method_6.to_excel(f'{result_dir_path}/Method_6.xlsx')
df_method_7.to_excel(f'{result_dir_path}/Method_7.xlsx')
df_method_8.to_excel(f'{result_dir_path}/Method_8.xlsx')
bar.finish()
print('Simulation done')

Time:  0:05:18 Elapsed Time: 0:05:18  (100%)   Country: United Kingdom Running: Ploting result: Death factor                                                   


Simulation done


In [11]:
columns = ['covid cases by ifr gamma=0 (fraction of population)[%]']

d = {'method 1': df_method_1.loc[:, columns],
     'method 2': df_method_2.loc[:, columns],
     'method 3': df_method_3.loc[:, columns],
     'method 4': df_method_4.loc[:, columns],
     'method 5': df_method_5.loc[:, columns],
     'method 6': df_method_6.loc[:, columns],
     'method 7': df_method_7.loc[:, columns],
     'method 8': df_method_8.loc[:, columns],}
pd.concat(d, axis=1).to_excel(f'{result_dir_path}/gamma=0.xlsx')

columns = ['covid cases by ifr gamma=0.8 (fraction of population)[%]']

d = {'method 1': df_method_1.loc[:, columns],
     'method 2': df_method_2.loc[:, columns],
     'method 3': df_method_3.loc[:, columns],
     'method 4': df_method_4.loc[:, columns],
     'method 5': df_method_5.loc[:, columns],
     'method 6': df_method_6.loc[:, columns],
     'method 7': df_method_7.loc[:, columns],
     'method 8': df_method_8.loc[:, columns],}
pd.concat(d, axis=1).to_excel(f'{result_dir_path}/gamma=0.8.xlsx')

columns = ['covid cases by ifr gamma=1 (fraction of population)[%]']

d = {'method 1': df_method_1.loc[:, columns],
     'method 2': df_method_2.loc[:, columns],
     'method 3': df_method_3.loc[:, columns],
     'method 4': df_method_4.loc[:, columns],
     'method 5': df_method_5.loc[:, columns],
     'method 6': df_method_6.loc[:, columns],
     'method 7': df_method_7.loc[:, columns],
     'method 8': df_method_8.loc[:, columns],}
pd.concat(d, axis=1).to_excel(f'{result_dir_path}/gamma=1.xlsx')

columns = ['covid cases by ifr gamma=0 (fraction of population)[%]', 'covid cases by ifr gamma=0.8 (fraction of population)[%]', 'covid cases by ifr gamma=1 (fraction of population)[%]']

d = {'method 1': df_method_1.loc[:, columns],
     'method 2': df_method_2.loc[:, columns],
     'method 3': df_method_3.loc[:, columns],
     'method 4': df_method_4.loc[:, columns],
     'method 5': df_method_5.loc[:, columns],
     'method 6': df_method_6.loc[:, columns],
     'method 7': df_method_7.loc[:, columns],
     'method 8': df_method_8.loc[:, columns],}
pd.concat(d, axis=1).to_excel(f'{result_dir_path}/gamma=all.xlsx')